In [13]:
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage

In [14]:
import polars as pl
import requests
from io import BytesIO
from typing import Dict, Any # type: ignore
from langchain_core.tools import tool

@tool(description="get_information_from_url")
def get_information_from_url(url: str) -> Dict[str, Any]:
    """
    Lấy dữ liệu báo cáo hàng ngày từ Google Sheet bằng URL
    """
    # Tải dữ liệu dưới dạng bytes
    sheet_id = url.split("/")[-2]
    csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

    response = requests.get(csv_url)
    response.raise_for_status()

    # Đọc CSV với encoding UTF-8
    try:
        df = pl.read_csv(BytesIO(response.content), encoding="utf8")
        df_formated = df.with_columns(pl.col('Date').str.to_datetime().dt.date().alias('date_formated')).sort(by='date_formated',descending=True) # type: ignore
        for content in df_formated.iter_rows():
            if content[1]:
                return df.filter(pl.col('Date')==content[0]).to_struct()[0]
    except Exception as e:
        print(e)
        return {}

In [15]:
# Cải thiện mô tả tool để rõ ràng hơn
@tool(description="Get information from Google Sheet URL")
def get_information_from_url(url: str) -> Dict[str, Any]:
    """
    Lấy dữ liệu báo cáo hàng ngày từ Google Sheet bằng URL.
    Nhận vào URL của Google Sheet và trả về dữ liệu mới nhất.
    Sử dụng tool này khi cần truy xuất dữ liệu từ Google Sheet.
    """
    # Tải dữ liệu dưới dạng bytes
    sheet_id = url.split("/")[-2]
    csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

    response = requests.get(csv_url)
    response.raise_for_status()

    # Đọc CSV với encoding UTF-8
    try:
        df = pl.read_csv(BytesIO(response.content), encoding="utf8")
        df_formated = df.with_columns(pl.col('Date').str.to_datetime().dt.date().alias('date_formated')).sort(by='date_formated',descending=True)
        for content in df_formated.iter_rows():
            if content[1]:
                return df.filter(pl.col('Date')==content[0]).to_struct()[0]
    except Exception as e:
        print(e)
        return {}

# Tạo LLM với temperature thấp hơn để tăng khả năng gọi tool
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
llm_with_tools = llm.bind_tools([get_information_from_url])

# Làm rõ prompt để LLM hiểu cần sử dụng tool
url = "https://docs.google.com/spreadsheets/d/19zPaWbW-VuYqhq1ur8vcPoV4EbDKKVDDN84vi4yuOJU/edit?usp=sharing"
query = f"Sử dụng tool get_information_from_url để lấy báo cáo từ URL: {url}"

# Gọi LLM
ai_msg = llm_with_tools.invoke(query)

# Kiểm tra tool calls
print("Tool calls:", ai_msg.tool_calls)

# Xử lý kết quả an toàn
if ai_msg.tool_calls:
    tool_message = ToolMessage(
        content=get_information_from_url(ai_msg.tool_calls[0]["args"]["url"]),
        tool_call_id=ai_msg.tool_calls[0]["id"],
    )
    final_response = llm_with_tools.invoke([ai_msg, tool_message])
    print("Final response:", final_response.content)
else:
    print("Không có tool nào được gọi. Hãy thử điều chỉnh prompt hoặc mô tả tool.")

Tool calls: [{'name': 'get_information_from_url', 'args': {'url': 'https://docs.google.com/spreadsheets/d/19zPaWbW-VuYqhq1ur8vcPoV4EbDKKVDDN84vi4yuOJU/edit?usp=sharing'}, 'id': '67abc3ea-02d1-48eb-a97b-a50bd1374131', 'type': 'tool_call'}]
Final response: OK. I have the information from the Google Sheet. The date is 15/06/2025. The tasks in progress are "Nghiên cứu finetuning_model, LoRA, QLoRA". There are no blockers. The completed task is "Time optimization for analysis intent".
